# CBOW Implementation of Word2Vec

This is part of the background research that I'm working on for [viberary.pizza](https://viberary.pizza/).

## Background 

[Word2vec](https://arxiv.org/abs/1301.3781) was a critical point in NLP work, building on previous work in dimensionality reduction in NLP such as tf-idf, topic modeling, and latent semantic analysis to reduce vocabulary sizes for computational complexity, and additionally, to add context by embedding similar words in the same latent space. 

As of 2022, it's almost been superceded by [transformers-based architectures](https://e2eml.school/transformers.html), but it's still worth understanding how it works in a historical context, as well as because there is a fair amount of it [in production in Spark](https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.ml.feature.Word2Vec.html).  

# Word2Vec Implementation

There are numerous word2vec implementations in libraries like Spark and Tensorflow. There is not an exact one in PyTorch, but following[this code](https://github.com/FraLotito/pytorch-continuous-bag-of-words/blob/master/cbow.py), as well as reading about the [architecture here](https://towardsdatascience.com/word2vec-with-pytorch-implementing-original-paper-2cd7040120b0) and [here](https://jalammar.github.io/illustrated-word2vec/),  I was able to implement and understand how it works under te covers


Original explanation in [PyTorch implementation](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html) is here. 

As a starting point, we take our raw data for input. 
Our [training set for Viberary is some sample the Goodreads input dataset](https://github.com/veekaybee/viberary#input-data-sample), 
which is a string of text containing the metadata for each unique book
id. 

For a single book id, it will contain the book description book title, etc. So a sample of a single book, will look like this


```
Raw text: All's Fairy in Love and War (Avalon: Web of Magic, #8) To Kara's astonishment, she discovers that a portal has opened in her bedroom closet and two goblins 
have fallen through! They refuse to return to the fairy realms and be drafted for an impending war. 
In an attempt to roust the pesky creatures, Kara falls through the portal, smack into the middle of a huge war.
Kara meets Queen Selinda, who appoints Kara as a Fairy Princess and assigns her an impossible task: 
to put an end to the war using her diplomatic skills.
```

This is initially stored as a Python string. 

Our final goal in learning a Word2Vec model with CBOW is, given an input phrase over a context window, to predict the word that's missing. The context window is how many words before and after the word we care about. So, given the phrase "Kara falls X the portal", we should be able to predict that the correct word is "through."

We do this in Word2Vec by continuously sampling from the raw text over the context window, where the context window around the word is the X variable and the word itself is the target variable. 

For the first example, "Kara falls the portal" is the context and "through" is the response variable. Then we shift the window by 1 word and generate another entry. This is the whole of the [continuous bag of words approach.](https://arxiv.org/pdf/1301.3781.pdf)

When we're first training the model, we pass these samples into the model and ask it to make a prediction on a single word given all these samples. The output is a vector of propabilities of the sample related to each word. We then compare that prediction to the actual label (I.e. for the sample "Kara falls X the portal" we KNOW the correct word is "through").

We compare the actual vector (i.e. where through = 1) to the probability vector, and the difference between the two is the loss. The parameters are passed to the model across multiple epochs and continuously updated until we minimize the loss, i.e. we get as close to the predicted word as possible. 

In the process of doing this prediction, we create a lookup table of words, or embeddings matrix, to their vector representations. It is these vectors that become our embeddings. Andiamo!

In [2]:
!pip install -q torch  # if you don't have it already


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [3]:
# we need these bois
import torch
import torch.nn as nn

/Users/vicki/viberary/viberary/neuron/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Text Preparation

In [5]:
# First, we'll initialize our hyperparameters for the model: 

CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right - this is our context window
EMBEDDING_DIM = 100  # size of the embeddings matrix - we'll get to this in a bit

In [6]:
# Our tiny training dataset 

raw_text = """To Kara's astonishment, she discovers that a portal has opened in her bedroom closet and two goblins have fallen through! They refuse to return to the fairy realms and be drafted for an impending war. 
In an attempt to roust the pesky creatures, Kara falls through the portal, 
smack into the middle of a huge war. Kara meets Queen Selinda, who appoints 
Kara as a Fairy Princess and assigns her an impossible task: 
to put an end to the war using her diplomatic skills.""".split()

In [7]:
# Text preprocessing get only individual words
vocab = set(raw_text)  # dedup
vocab_size = len(vocab)

In [8]:
print(vocab)

{'Princess', 'the', 'a', 'to', 'put', 'be', 'portal', 'realms', 'creatures,', 'Selinda,', 'that', 'Fairy', 'skills.', 'bedroom', 'refuse', 'pesky', 'through!', 'using', 'discovers', 'into', 'portal,', 'war', 'smack', 'has', 'have', 'end', 'closet', "Kara's", 'meets', 'In', 'drafted', 'They', 'task:', 'middle', 'attempt', 'who', 'assigns', 'and', 'diplomatic', 'Queen', 'she', 'of', 'as', 'two', 'impossible', 'her', 'goblins', 'fairy', 'roust', 'for', 'opened', 'an', 'falls', 'through', 'impending', 'Kara', 'appoints', 'astonishment,', 'in', 'fallen', 'huge', 'return', 'To', 'war.'}


In [9]:
# we create simple mappings of word to an index of the word
word_to_ix = {word: ix for ix, word in enumerate(vocab)}
ix_to_word = {ix: word for ix, word in enumerate(vocab)}

In [10]:
print(word_to_ix)

{'Princess': 0, 'the': 1, 'a': 2, 'to': 3, 'put': 4, 'be': 5, 'portal': 6, 'realms': 7, 'creatures,': 8, 'Selinda,': 9, 'that': 10, 'Fairy': 11, 'skills.': 12, 'bedroom': 13, 'refuse': 14, 'pesky': 15, 'through!': 16, 'using': 17, 'discovers': 18, 'into': 19, 'portal,': 20, 'war': 21, 'smack': 22, 'has': 23, 'have': 24, 'end': 25, 'closet': 26, "Kara's": 27, 'meets': 28, 'In': 29, 'drafted': 30, 'They': 31, 'task:': 32, 'middle': 33, 'attempt': 34, 'who': 35, 'assigns': 36, 'and': 37, 'diplomatic': 38, 'Queen': 39, 'she': 40, 'of': 41, 'as': 42, 'two': 43, 'impossible': 44, 'her': 45, 'goblins': 46, 'fairy': 47, 'roust': 48, 'for': 49, 'opened': 50, 'an': 51, 'falls': 52, 'through': 53, 'impending': 54, 'Kara': 55, 'appoints': 56, 'astonishment,': 57, 'in': 58, 'fallen': 59, 'huge': 60, 'return': 61, 'To': 62, 'war.': 63}


In [11]:
# Creating our training data and context window

def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)


data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = [raw_text[i - 2], raw_text[i - 1], raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))

In [12]:
# We have our [input, input, input, input, target]
# based on the context window of +2 words -2 words
# you can see how we're building words close to each other now
print(*data[0:10], sep="\n")

(['To', "Kara's", 'she', 'discovers'], 'astonishment,')
(["Kara's", 'astonishment,', 'discovers', 'that'], 'she')
(['astonishment,', 'she', 'that', 'a'], 'discovers')
(['she', 'discovers', 'a', 'portal'], 'that')
(['discovers', 'that', 'portal', 'has'], 'a')
(['that', 'a', 'has', 'opened'], 'portal')
(['a', 'portal', 'opened', 'in'], 'has')
(['portal', 'has', 'in', 'her'], 'opened')
(['has', 'opened', 'her', 'bedroom'], 'in')
(['opened', 'in', 'bedroom', 'closet'], 'her')


## Model Set Up
# CBOW Architecture 

<img width="344" alt="Screen Shot 2023-02-14 at 3 48 16 PM" src="https://user-images.githubusercontent.com/3837836/218859716-495a0a6f-aed7-40aa-aba9-5c0f1949788c.png">

We have two layers in the CBOW implementation of Word2Vec: an input Embedding layer that maps each word to a space in the embedding dictionary, a hidden linear activation layer, and then the output layer that is the proportional probabilities [softmax](https://en.wikipedia.org/wiki/Softmax_function) of all the correct words given an input window. 

The critical part is the first part, creating the Embeddings lookup. 

First, we associate each word in the vocabulary with an index, aka `{'she': 0, 'middle': 1, 'put': 2`

Then, what we want to do is create an embeddings table, or matrix, that we will multiply with these indices to map each one to its correct place in relation to the other indices via a table lookup, based on how many vectors you'd like to represent the word. 

There is a [really good explanation](https://stats.stackexchange.com/questions/270546/how-does-keras-embedding-layer-work/305032#305032) of how these are generated: 

``` 
For a given word, you create a one-hot vector based on its index and multiply it by the embeddings matrix, effectively replicating a lookup. For instance, for the word "soon" the index is 4, and the one-hot vector is [0, 0, 0, 0, 1, 0, 0]. If you multiply this (1, 7) matrix by the (7, 2) embeddings matrix you get the desired two-dimensional embedding, which in this case is [2.2, 1.4].
```

In [13]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):  # we pass in vocab_size and embedding_dim as hyperparams
        super(CBOW, self).__init__()

        # out: 1 x embedding_dim
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)  # initialize an Embedding matrix based on our inputs
        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()

        # out: 1 x vocab_size
        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim=-1)

    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1, -1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        # Embeddings lookup of a single word once the Embeddings layer has been optimized 
        return self.embeddings(word).view(1, -1)

In [14]:
# We initialize the model:

model = CBOW(vocab_size, EMBEDDING_DIM)

In [15]:
# then, we initialize the loss function 
# (aka how close our predicted word is to the actual word and how we want to minimize it using the optimizer)

loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

In [16]:
# Training

# 50 to start with, no correct answer here
for epoch in range(50):
    # we start tracking how accurate our intial words are
    total_loss = 0

    # for the x, y in the training data: 
    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)

        # we look at loss
        log_probs = model(context_vector)

        # we compare the loss from what the actual word is related to the probaility of the words
        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]))

    # optimize at the end of each epoch
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

    # Log out some metrics to see if loss decreases
    print("end of epoch {} | loss {:2.3f}".format(epoch, total_loss))

end of epoch 0 | loss 339.036
end of epoch 1 | loss 326.887
end of epoch 2 | loss 315.408
end of epoch 3 | loss 304.454
end of epoch 4 | loss 293.905
end of epoch 5 | loss 283.696
end of epoch 6 | loss 273.754
end of epoch 7 | loss 264.069
end of epoch 8 | loss 254.643
end of epoch 9 | loss 245.419
end of epoch 10 | loss 236.401
end of epoch 11 | loss 227.571
end of epoch 12 | loss 218.917
end of epoch 13 | loss 210.448
end of epoch 14 | loss 202.127
end of epoch 15 | loss 193.950
end of epoch 16 | loss 185.895
end of epoch 17 | loss 177.959
end of epoch 18 | loss 170.169
end of epoch 19 | loss 162.515
end of epoch 20 | loss 155.011
end of epoch 21 | loss 147.695
end of epoch 22 | loss 140.536
end of epoch 23 | loss 133.539
end of epoch 24 | loss 126.737
end of epoch 25 | loss 120.130
end of epoch 26 | loss 113.735
end of epoch 27 | loss 107.553
end of epoch 28 | loss 101.606
end of epoch 29 | loss 95.903
end of epoch 30 | loss 90.437
end of epoch 31 | loss 85.233
end of epoch 32 | los

In [17]:
# Now, let's test to see if the model predicts the correct word using our initial input
context = ["Kara", "falls", "the", "portal"]
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

In [18]:
print(f'Raw text: {" ".join(raw_text)}\n')
print(f"Context: {context}\n")
print(f"Prediction: {ix_to_word[torch.argmax(a[0]).item()]}")


Raw text: To Kara's astonishment, she discovers that a portal has opened in her bedroom closet and two goblins have fallen through! They refuse to return to the fairy realms and be drafted for an impending war. In an attempt to roust the pesky creatures, Kara falls through the portal, smack into the middle of a huge war. Kara meets Queen Selinda, who appoints Kara as a Fairy Princess and assigns her an impossible task: to put an end to the war using her diplomatic skills.

Context: ['Kara', 'falls', 'the', 'portal']

Prediction: creatures,


In [19]:

# Now let's get what we care about, which is the embeddings!
print(f'Getting vectors for a sequence:\n', model.embeddings(torch.LongTensor([1, 2, 3])))

Getting vectors for a sequence:
 tensor([[-7.1156e-02, -4.1980e-01,  7.5448e-01, -1.0346e+00,  9.7516e-01,
          4.3864e-01, -1.5752e+00,  2.3797e-01,  1.1792e+00,  2.7997e+00,
          3.3248e-01,  1.3918e+00,  7.5965e-01, -2.6596e-01,  1.6388e+00,
          8.9776e-01, -2.6370e+00,  4.8947e-01, -7.4175e-01, -6.0856e-01,
          2.0947e+00,  8.5964e-01, -3.2345e-01,  1.9920e+00,  8.7369e-01,
         -1.2016e+00,  1.8309e+00,  7.7124e-01,  6.3596e-01,  7.0087e-01,
         -2.6163e+00, -8.4904e-01, -1.8401e+00, -8.2744e-01,  2.4925e-01,
         -3.4602e-01,  6.5291e-01,  6.1475e-01, -4.9659e-01,  1.4916e+00,
          4.3811e-01, -9.9531e-01,  8.9120e-01, -5.7321e-02,  2.8509e-01,
         -4.1099e-01, -1.2889e+00, -9.1033e-01,  2.6394e-01,  1.3394e+00,
          1.7379e-01, -1.6040e+00,  6.0400e-01,  6.4703e-01,  9.8809e-01,
         -5.3346e-01,  5.5176e-01, -8.3570e-01, -1.3796e-02,  6.0779e-01,
          1.2596e+00,  1.4880e-01, -9.9366e-01,  1.9915e+00,  2.5511e+00,
     

In [20]:
print('Getting weights:\n', model.embeddings.weight.data[1])  # we can get the entire matrix this way

Getting weights:
 tensor([-0.0712, -0.4198,  0.7545, -1.0346,  0.9752,  0.4386, -1.5752,  0.2380,
         1.1792,  2.7997,  0.3325,  1.3918,  0.7596, -0.2660,  1.6388,  0.8978,
        -2.6370,  0.4895, -0.7418, -0.6086,  2.0947,  0.8596, -0.3235,  1.9920,
         0.8737, -1.2016,  1.8309,  0.7712,  0.6360,  0.7009, -2.6163, -0.8490,
        -1.8401, -0.8274,  0.2493, -0.3460,  0.6529,  0.6147, -0.4966,  1.4916,
         0.4381, -0.9953,  0.8912, -0.0573,  0.2851, -0.4110, -1.2889, -0.9103,
         0.2639,  1.3394,  0.1738, -1.6040,  0.6040,  0.6470,  0.9881, -0.5335,
         0.5518, -0.8357, -0.0138,  0.6078,  1.2596,  0.1488, -0.9937,  1.9915,
         2.5511,  1.0531, -1.1937,  0.9104, -2.2801, -0.2374,  1.1270,  0.2845,
        -0.8112,  0.3993, -0.7435,  0.7523,  0.1413,  0.7285,  0.3239,  0.3489,
        -0.1597,  0.3593, -2.5938, -0.1778, -1.8794, -1.5973,  1.1407,  1.5752,
        -0.0087,  0.3636,  1.2005,  0.6010,  1.0035, -0.8222, -1.0813,  0.9766,
        -0.9388, -0.87

In [21]:
# And, what we actually care about is being able to look up individual words with their embeddings: 
torch.set_printoptions(threshold=10_000)
print(f"Embedding for Kara: {model.embeddings.weight[word_to_ix['Kara']]}")

Embedding for Kara: tensor([-0.0529, -0.1168,  2.1033, -0.8198, -0.4394,  1.5762,  0.2513,  1.2508,
        -0.9334, -1.1929, -0.1158, -0.5309, -0.4978,  1.1619,  0.0430, -0.3920,
         1.0307, -0.7650,  0.1039, -0.4137,  0.3649, -0.3316,  1.0523,  0.7373,
         0.9990, -1.1825,  0.9387, -1.0296, -0.4185, -0.6773,  1.5025, -0.5155,
         0.8870,  1.2185, -0.6938,  1.0994,  0.2009,  0.1723,  0.3883, -0.9892,
        -0.2323, -0.4497, -0.9362, -1.2295, -0.0628,  1.9673,  0.4310, -0.9807,
        -1.8765, -1.1365,  1.1282,  1.3473, -1.0122, -1.6675,  0.3852, -0.5562,
        -0.2671,  0.2921, -1.0951,  0.3297,  1.1109, -1.7162,  1.8813,  0.1143,
        -1.6262,  0.1501, -1.8283,  1.2635,  0.2079, -0.7766,  0.7787,  0.2996,
         0.1116,  0.6562, -0.3903,  1.0429, -0.1710, -0.1728, -0.7672, -1.4438,
        -0.3760,  0.8299,  1.7041, -0.6659,  0.8116,  1.4468, -1.0999, -2.1328,
        -1.6577, -0.3977, -0.6104,  0.1883, -0.0971, -0.7148,  0.8364,  0.0606,
         0.8489,  1.

In [22]:
# This way, when we create our second tower of book words, we know which ones are likely related to a given book